In [63]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import branca.colormap as cmp
import folium
import shapely
import plotly.express as px

In [64]:
# Canada, Quebec City Dataframe (Main City)
#df, summary data frame
df=pd.read_csv('../data/canada/listings.csv')
#dfd, detailed data frame
dfd=pd.read_csv('../data/canada/detailed_listing.csv')

In [65]:
# Remove coloumn 'neighbourhood_group' and 'license'  because it's all NaN
df.drop(['neighbourhood_group','license'], axis=1, inplace=True)
# Show the shape
df.shape

(13621, 16)

In [66]:
# Sample 10 Rows (Summary)
df.head(10)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,298059,Superb cottage Mile-End Plateau !,1536474,Helene,Le Plateau-Mont-Royal,45.52665,-73.60730,Entire home/apt,180,30,52,2014-10-18,0.40,1,104,0
1,2843,Fabulous downtown/airport room,2319,Gail,Le Sud-Ouest,45.48496,-73.58001,Private room,60,30,157,2022-08-21,0.97,2,80,5
2,309367,Charming and Clean ! 5 min Métro,1562348,Virginie,Villeray-Saint-Michel-Parc-Extension,45.55169,-73.60007,Entire home/apt,64,180,22,2015-08-13,0.18,3,74,0
3,29059,Lovely studio Quartier Latin,125031,Maryline,Ville-Marie,45.51939,-73.56482,Entire home/apt,152,2,385,2022-09-02,2.58,2,306,54
4,29061,Maison historique - Quartier Latin,125031,Maryline,Ville-Marie,45.51929,-73.56493,Entire home/apt,286,3,108,2022-08-29,0.84,2,250,27
5,331764,Loft sur le Plateau avec terrasse,1676310,Julien,Le Plateau-Mont-Royal,45.52916,-73.57174,Entire home/apt,45,30,63,2022-08-11,0.52,1,90,2
6,34715,Chez Patrac ! Montreal - Métro Beaubien,149769,Alain,Rosemont-La Petite-Patrie,45.53818,-73.60360,Entire home/apt,160,14,8,2022-07-26,0.06,1,117,4
7,344508,Convenient location and comfortable,839115,Nazzarena,LaSalle,45.44160,-73.62942,Private room,33,30,6,2012-10-10,0.05,1,0,0
8,36301,Romantic & peaceful Plateau loft,381468,Sylvie,Le Plateau-Mont-Royal,45.53026,-73.58413,Entire home/apt,160,31,45,2022-08-22,0.51,6,323,3
9,353210,Small room in a wonderful house,1789044,Colin&Özge,Mercier-Hochelaga-Maisonneuve,45.55646,-73.53311,Private room,32,31,26,2022-08-31,0.22,3,255,2


In [67]:
# Sample 10 Rows (Detailed)
dfd.head(10)
dfd.shape

(13621, 75)

In [68]:
# Correlation heatmap for summary
df.corr().style.background_gradient(cmap='coolwarm')

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
id,1.000000,0.382055,-0.041054,0.002701,0.011799,-0.030496,-0.213334,0.418446,0.059735,0.275619,-0.092137
host_id,0.382055,1.000000,-0.061506,-0.017911,0.020988,-0.039192,-0.099428,0.351490,0.042615,0.146561,0.133921
latitude,-0.041054,-0.061506,1.000000,0.234728,-0.014075,-0.015795,-0.001435,-0.059956,-0.104928,-0.021720,-0.032080
longitude,0.002701,-0.017911,0.234728,1.000000,0.012905,-0.003283,0.040691,0.046751,0.056574,-0.070911,0.045261
price,0.011799,0.020988,-0.014075,0.012905,1.000000,0.072424,0.000599,0.053258,0.003217,0.032782,0.006715
minimum_nights,-0.030496,-0.039192,-0.015795,-0.003283,0.072424,1.000000,-0.027753,-0.055129,0.002918,0.043878,-0.053743
number_of_reviews,-0.213334,-0.099428,-0.001435,0.040691,0.000599,-0.027753,1.000000,0.391604,-0.034509,0.057025,0.641430
reviews_per_month,0.418446,0.351490,-0.059956,0.046751,0.053258,-0.055129,0.391604,1.000000,0.084338,0.194895,0.689412
calculated_host_listings_count,0.059735,0.042615,-0.104928,0.056574,0.003217,0.002918,-0.034509,0.084338,1.000000,0.052988,0.038390
availability_365,0.275619,0.146561,-0.021720,-0.070911,0.032782,0.043878,0.057025,0.194895,0.052988,1.000000,0.135132


In [69]:
# Correlation heatmap for detailed
dfd.corr().style.background_gradient(cmap='coolwarm')

/opt/conda/lib/python3.9/site-packages/pandas/io/formats/style.py:3554: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/opt/conda/lib/python3.9/site-packages/pandas/io/formats/style.py:3555: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,1.000000,nan,0.382055,0.041387,0.044995,nan,-0.041054,0.002701,0.041595,nan,0.014655,0.004572,-0.030496,-0.129060,-0.068576,-0.031561,-0.015103,-0.020142,-0.039306,-0.020142,nan,0.306715,0.334627,0.339828,0.275619,-0.213334,-0.092137,0.167484,0.007738,-0.087484,-0.066315,-0.065395,-0.062384,-0.065617,-0.110251,0.059735,0.029316,0.143463,-0.021397,0.418446
scrape_id,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
host_id,0.382055,nan,1.000000,-0.021572,-0.011267,nan,-0.061506,-0.017911,0.046876,nan,0.006010,0.015615,-0.039192,-0.057905,-0.088982,-0.041729,0.011704,0.028495,-0.052456,0.028479,nan,0.245674,0.275877,0.276423,0.146561,-0.099428,0.133921,0.190712,-0.067556,-0.156256,-0.133051,-0.132307,-0.158898,-0.095134,-0.174316,0.042615,-0.009832,0.230107,0.009736,0.351490
host_listings_count,0.041387,nan,-0.021572,1.000000,0.977080,nan,-0.081244,0.036880,-0.030204,nan,-0.023622,-0.040626,0.009487,0.022961,0.010165,0.009398,-0.003266,-0.004356,0.014488,-0.004356,nan,0.140761,0.168372,0.171616,-0.010963,-0.044811,-0.016610,-0.032252,-0.057202,-0.132678,-0.080724,-0.096942,-0.111005,-0.044968,-0.165585,0.833924,0.837162,0.098187,-0.003020,0.032863
host_total_listings_count,0.044995,nan,-0.011267,0.977080,1.000000,nan,-0.081030,0.042930,-0.024425,nan,-0.018710,-0.036116,0.007254,0.018655,0.006640,0.007951,-0.002793,-0.003103,0.012550,-0.003103,nan,0.138601,0.165231,0.167473,0.001278,-0.029258,0.011503,-0.018637,-0.052741,-0.129238,-0.076513,-0.101262,-0.116418,-0.038176,-0.165105,0.770743,0.772368,0.097183,-0.005615,0.071440
neighbourhood_group_cleansed,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
latitude,-0.041054,nan,-0.061506,-0.081244,-0.081030,nan,1.000000,0.234728,-0.022000,nan,-0.013362,-0.009284,-0.015795,-0.020755,-0.017669,-0.005617,-0.012732,-0.020096,-0.012919,-0.020092,nan,-0.079936,-0.083358,-0.078987,-0.021720,-0.001435,-0.032080,-0.024841,0.006943,0.054427,0.047697,0.037749,0.035285,-0.050231,0.051109,-0.104928,-0.084821,-0.097919,-0.024620,-0.059956
longitude,0.002701,nan,-0.017911,0.036880,0.042930,nan,0.234728,1.000000,-0.048435,nan,-0.100327,-0.076842,-0.003283,0.006518,-0.020034,0.002093,0.013425,0.019753,-0.002535,0.019751,nan,-0.026595,-0.025061,-0.026293,-0.070911,0.040691,0.045261,0.023519,-0.017281,-0.014872,-0.024998,-0.030019,-0.027866,0.074847,-0.016547,0.056574,0.056704,0.003452,0.001470,0.046751
accommodates,0.041595,nan,0.046876,-0.030204,-0.024425,nan,-0.022000,-0.048435,1.000000,nan,0.745164,0.802087,-0.021194,0.018087,-0.039782,-0.017388,-0.000292,-0.004040,-0.026512,-0.004035,nan,0.179129,0.174495,0.169533,0.101942,0.123572,0.136082,0.123293,0.034323,-0.007235,0.027267,0.008793,-0.002323,-0.001956,-0.012238,-0.015204,0.010778,-0.108452,-0.041006,0.127095
bathrooms,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [70]:
# review score correlation
dfr = dfd.iloc[:,61:67]
dfr.corr()

,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location
review_scores_rating,1.000000,0.830344,0.802334,0.682693,0.732341,0.561838
review_scores_accuracy,0.830344,1.000000,0.730043,0.674302,0.710032,0.567963
review_scores_cleanliness,0.802334,0.730043,1.000000,0.575513,0.597799,0.466767
review_scores_checkin,0.682693,0.674302,0.575513,1.000000,0.767861,0.513426
review_scores_communication,0.732341,0.710032,0.597799,0.767861,1.000000,0.531904
review_scores_location,0.561838,0.567963,0.466767,0.513426,0.531904,1.000000


In [71]:
# 1/3 Sorts neighborhood mean price 
neig = df[['price','neighbourhood']].groupby('neighbourhood').mean()
neig = neig.sort_values('price')
neig['rank'] = np.arange(len(neig)) + 1
print(neig)

                                               price  rank
neighbourhood                                             
Montréal-Est                               73.000000     1
Montréal-Ouest                             87.100000     2
Anjou                                      87.415094     3
Villeray-Saint-Michel-Parc-Extension       97.604575     4
Ahuntsic-Cartierville                     110.733042     5
Verdun                                    113.399317     6
LaSalle                                   115.560606     7
Sainte-Anne-de-Bellevue                   117.105263     8
Beaconsfield                              118.476190     9
Mont-Royal                                125.173913    10
Montréal-Nord                             128.672727    11
Côte-des-Neiges-Notre-Dame-de-Grâce       128.739346    12
Pointe-Claire                             129.075000    13
Rosemont-La Petite-Patrie                 132.083485    14
Saint-Laurent                             135.157609    

In [72]:
# 2/3 only run once or it will keep on creating indexes
neig = neig.reset_index(level=0)


In [73]:
# 3/3 The Column
print(neig)

                               neighbourhood       price  rank
0                               Montréal-Est   73.000000     1
1                             Montréal-Ouest   87.100000     2
2                                      Anjou   87.415094     3
3       Villeray-Saint-Michel-Parc-Extension   97.604575     4
4                      Ahuntsic-Cartierville  110.733042     5
5                                     Verdun  113.399317     6
6                                    LaSalle  115.560606     7
7                    Sainte-Anne-de-Bellevue  117.105263     8
8                               Beaconsfield  118.476190     9
9                                 Mont-Royal  125.173913    10
10                             Montréal-Nord  128.672727    11
11       Côte-des-Neiges-Notre-Dame-de-Grâce  128.739346    12
12                             Pointe-Claire  129.075000    13
13                 Rosemont-La Petite-Patrie  132.083485    14
14                             Saint-Laurent  135.15760

In [74]:
# Bar plot neighborhood mean sorted (draft v2)
plt.bar(*zip(*neig.sort_values(ascending=False).items())) + plt.xticks(rotation=90)
plt.show


TypeError: sort_values() missing 1 required positional argument: 'by'

In [ ]:
# Set bounds
BBox = ((df.longitude.min(),   df.longitude.max(), df.latitude.min(), df.latitude.max()))
# Import Image
ruh_m = plt.imread('../data/canada/map.png')
# Plot Data on Map (Draft v1)
fig, ax = plt.subplots(figsize = (8,7))
ax.scatter(df.longitude, df.latitude, zorder=1, alpha= 0.2, c='b', s=10)
ax.set_title('Plotting Spatial Data on Montreal Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')


In [ ]:
#linear = cmp.LinearColormap(['Blue', 'Red'], vmin = df.price.min(), vmax=df.price.max())

In [ ]:
# Bar plot of room types' mean price sorted (Working on)
plot3=sns.barplot(x='room_type', y='price',data=df)
plot3.set(xlabel='Room Type', ylabel='Average Price',title='Room Type Average Price in Montreal')
plt.show()